In [28]:
import sys
from multiprocessing import Pool, cpu_count

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn
import torch.utils.data as data_utils
from tqdm import tqdm

sys.path.append('../')
from wiki import val, rnn, rnn_predict
from wiki.newphet import prophet_plot
from wiki.utils import clock

%matplotlib inline

In [4]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [5]:
batch_size = 512
pred_len = 60

In [98]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_1.csv').fillna(0)

In [99]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [10]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [11]:
save_best_path = base_dir+'rnn_v2_lr2.mdl'
model = torch.load(save_best_path).cuda()

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'wiki.rnn.RNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [100]:
outputs, targets, sequences = model.predict(valloader)

In [101]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [102]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_1.csv')
X = train_df.drop('Page', axis=1).values

In [103]:
predictions = scaler.inverse_transform(predictions.T).T
true = X

In [130]:
    def smape(y_true, y_pred):
        # NOTE: should check and make sure that NaNs aren't included
        denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
        diff = np.abs(y_true - y_pred) / denominator
        diff[denominator == 0] = 0.0
        return np.nanmean(diff, axis=1)

In [141]:
with clock():
    smapes = smape(true[:,-60:], predictions[:,-60:])
    smapes_clipped = smape(true[:,-60:], predictions[:,-60:].round().clip(0))

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  
/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


Elapsed time 0.4242696762084961 seconds


In [132]:
smapes

array([ 39.95109301,  42.55102878,  49.62733476, ...,          nan,
                nan,          nan])

In [133]:
smapes.shape

(145063,)

In [134]:
np.nanmean(smapes), np.nanmean(smapes)

47.409468129657554